### FAISS

- Facebook AI Similarity Search (FAISS) is a library for efficient similarity search and clustering of dense vectors.
- It contains algorithm that search in sets of vectors of any size, up to ones that possibility do not fit in RAM.
- It also contains supporting code for evalution and parameter tuning

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
docs = text_splitter.split_documents(documents)

In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [3]:
embedding_model = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(docs, embedding_model)

C:\Users\vaibh\AppData\Local\Temp\ipykernel_19044\323140630.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="gemma:2b")


In [4]:
# Query 
query = "How does the speaker describe the desired outcome of the war?"
qr = db.similarity_search(query)
qr

[Document(id='6ad4e10e-28b3-496b-bef0-87f41f7674b0', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7881e9df-72ee-437a-9720-13f0d4f70605', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. Ther

#### As a Retriver

- We can also convert the vectorstore into a Retriver class.
- This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [5]:
retriver = db.as_retriever()
docs = retriver.invoke(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score

- similrity_search_with_score allows us to return not only the documents but also the distance score of the query to them.
- The returned distance score is L2 distance.
- Therefore, a lower score is better

In [6]:
docs_and_score =  db.similarity_search_with_score(query)
docs_and_score

[(Document(id='6ad4e10e-28b3-496b-bef0-87f41f7674b0', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2953.364)),
 (Document(id='7881e9df-72ee-437a-9720-13f0d4f70605', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed i

In [7]:
embedding_vector = embedding_model.embed_query(query)

docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='6ad4e10e-28b3-496b-bef0-87f41f7674b0', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7881e9df-72ee-437a-9720-13f0d4f70605', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. Ther

#### Saving and Loading

In [8]:
db.save_local('faiss_index')

In [9]:
new_db = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)


In [10]:
docs

[Document(id='6ad4e10e-28b3-496b-bef0-87f41f7674b0', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='7881e9df-72ee-437a-9720-13f0d4f70605', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. Ther